### Feature Store with Metadata and Versioning

#### Overview
This script sets up a **Feature Store** for storing feature metadata and values with versioning.

#### Steps

1. **Feature Metadata Management**
- Creates the `feature_store` table (if not exists).
- Stores metadata for selected features (`feature_name`, `feature_type`, `description`, `data_source`).
- Updates feature versions only if changes are detected.

2. **Fetching Latest Data Versions**
- Retrieves the latest **train** and **API** data versions from `transformed_train_data` and `transformed_api_data`.

3. **Feature Values Table**
- Creates the `feature_values` table (if not exists) with selected features.
- Stores feature values with a `data_source` (`train` or `api`) and **timestamp-based versioning**.

4. **Feature Ingestion**
- Inserts feature values for **train** and **API data**.
- API data does not include the `Exited` column.

#### Final Output
- **Feature metadata and values are stored in PostgreSQL with version tracking.**
**Feature Store setup completed!**


In [1]:
# Feature_store with metadata and versoning
import psycopg2
import pandas as pd

# Establish connection to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            dbname='bank_churn',
            user='postgres',
            password='131412aA@',
            host='localhost',
            port='5432'
        )
        return conn
    except Exception as e:
        print("Error connecting to database:", e)
        return None

# Create the feature_store table only if it doesn't exist (preserving existing data and versioning)
def create_feature_store_table_if_not_exists():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS feature_store (
                id SERIAL PRIMARY KEY,
                feature_name VARCHAR(255) UNIQUE,
                feature_type VARCHAR(50),  -- e.g., 'numeric', 'boolean'
                description TEXT,          -- A description of the feature
                data_source VARCHAR(10),   -- 'train' or 'api'
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                version INT DEFAULT 1,     -- Version number of the feature
                is_active BOOLEAN DEFAULT TRUE  -- If the feature is currently active
            );
        ''')
        conn.commit()
        cursor.close()
        conn.close()
        print("feature_store table is ready (if not already existing)!")

# Define the updated feature metadata for the selected features
feature_metadata = {
    'CreditScore': {'description': 'Customer credit score (normalized)', 'feature_type': 'numeric', 'data_source': 'train'},
    'Age': {'description': 'Age of the customer', 'feature_type': 'numeric', 'data_source': 'train'},
    'Tenure': {'description': 'Number of years with the bank', 'feature_type': 'numeric', 'data_source': 'train'},
    'Balance': {'description': 'Account balance of the customer', 'feature_type': 'numeric', 'data_source': 'train'},
    'NumOfProducts': {'description': 'Number of products used by the customer', 'feature_type': 'numeric', 'data_source': 'train'},
    'IsActiveMember': {'description': 'Customer activity status (1: Active, 0: Inactive)', 'feature_type': 'boolean', 'data_source': 'train'},
    'Geography_France': {'description': 'Indicator for French geography', 'feature_type': 'boolean', 'data_source': 'train'},
    'Geography_Germany': {'description': 'Indicator for German geography', 'feature_type': 'boolean', 'data_source': 'train'},
    'Geography_Spain': {'description': 'Indicator for Spanish geography', 'feature_type': 'boolean', 'data_source': 'train'},
    'BalancePerProduct': {'description': 'Balance per product', 'feature_type': 'numeric', 'data_source': 'train'},
    'Exited': {'description': 'Whether the customer exited (only available in train data)', 'feature_type': 'boolean', 'data_source': 'train'}
}

# Insert feature metadata into the feature_store table without affecting versioning unnecessarily
def insert_feature_metadata():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        for feature, details in feature_metadata.items():
            cursor.execute('''
                INSERT INTO feature_store (feature_name, feature_type, description, data_source, version)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT (feature_name) DO UPDATE 
                    SET version = feature_store.version + 1,
                        description = EXCLUDED.description,
                        feature_type = EXCLUDED.feature_type,
                        data_source = EXCLUDED.data_source
                WHERE feature_store.description IS DISTINCT FROM EXCLUDED.description
                   OR feature_store.feature_type IS DISTINCT FROM EXCLUDED.feature_type
                   OR feature_store.data_source IS DISTINCT FROM EXCLUDED.data_source;
            ''', (feature, details['feature_type'], details['description'], details['data_source'], 1))
        conn.commit()
        cursor.close()
        conn.close()
        print("Feature metadata inserted/updated successfully (only if changes detected)!")

# Retrieve and display feature metadata
def get_feature_metadata():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM feature_store')
        records = cursor.fetchall()
        cursor.close()
        conn.close()
        return pd.DataFrame(records, columns=['id', 'feature_name', 'feature_type', 'description', 'data_source', 'created_at', 'version', 'is_active'])

# Run the feature store setup without dropping the table every time
create_feature_store_table_if_not_exists()
insert_feature_metadata()

# Display stored feature metadata
print(get_feature_metadata())


feature_store table is ready (if not already existing)!
Feature metadata inserted/updated successfully (only if changes detected)!
    id       feature_name feature_type  \
0    1        CreditScore      numeric   
1    2                Age      numeric   
2    3             Tenure      numeric   
3    4            Balance      numeric   
4    5      NumOfProducts      numeric   
5    6     IsActiveMember      boolean   
6    7   Geography_France      boolean   
7    8  Geography_Germany      boolean   
8    9    Geography_Spain      boolean   
9   10  BalancePerProduct      numeric   
10  11             Exited      boolean   

                                          description data_source  \
0                  Customer credit score (normalized)       train   
1                                 Age of the customer       train   
2                       Number of years with the bank       train   
3                     Account balance of the customer       train   
4             Numbe

In [2]:
# Fetching the latest version of Transformed data
import pandas as pd

# Function to get the latest version from a table
def get_latest_version(table_name):
    conn = connect_db()
    latest_version = None
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute(f"SELECT MAX(version) FROM {table_name}")
            latest_version = cursor.fetchone()[0]  # Fetch latest timestamp
        except Exception as e:
            print(f"⚠️ Error fetching latest version from {table_name}:", e)
        finally:
            cursor.close()
            conn.close()
    return latest_version

# Get latest version from transformed tables
latest_train_version = get_latest_version("transformed_train_data")
latest_api_version = get_latest_version("transformed_api_data")

print(f"Latest Train Version: {latest_train_version}")
print(f"Latest API Version: {latest_api_version}")

# Function to count feature_values rows correctly
def count_feature_values():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        
        # Ensure that we filter by both version and data_source
        cursor.execute("SELECT COUNT(*) FROM feature_values WHERE version = %s AND data_source = 'train'", (latest_train_version,))
        train_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM feature_values WHERE version = %s AND data_source = 'api'", (latest_api_version,))
        api_count = cursor.fetchone()[0]

        cursor.close()
        conn.close()

        print(f"Feature Store Train Data Rows: {train_count}")
        print(f"Feature Store API Data Rows: {api_count}")

# Run the verification
count_feature_values()


Latest Train Version: 20250311_121740
Latest API Version: 20250311_121740
📊 Feature Store Train Data Rows: 165034
📊 Feature Store API Data Rows: 110023


In [3]:
# Creating Feature_values table
import psycopg2

# Function to connect to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            dbname='bank_churn',
            user='postgres',
            password='131412aA@',
            host='localhost',
            port='5432'
        )
        return conn
    except Exception as e:
        print("Error connecting to database:", e)
        return None

# Function to create feature_values table with correct schema
def create_feature_values_table():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS feature_values (
                id SERIAL PRIMARY KEY,
                CreditScore FLOAT,
                Age FLOAT,
                Tenure INT,
                Balance FLOAT,
                NumOfProducts INT,
                IsActiveMember INT,
                Geography_France BOOLEAN,
                Geography_Germany BOOLEAN,
                Geography_Spain BOOLEAN,
                BalancePerProduct FLOAT,
                Exited INT,  -- Nullable because API data won't have this
                data_source VARCHAR(10),  -- 'train' or 'api'
                version VARCHAR(20)  -- ✅ Corrected from INT to VARCHAR(20)
            );
        ''')
        conn.commit()
        cursor.close()
        conn.close()
        print("✅ feature_values table is ready!")

# Run table creation
create_feature_values_table()


✅ feature_values table is ready!


In [4]:
# feature ingestion into feature_values
import psycopg2
from datetime import datetime

# Function to connect to PostgreSQL
def connect_db():
    try:
        conn = psycopg2.connect(
            dbname='bank_churn',
            user='postgres',
            password='131412aA@',
            host='localhost',
            port='5432'
        )
        return conn
    except Exception as e:
        print("Error connecting to database:", e)
        return None

# Function to insert data into feature_values table
def insert_feature_values(data_source, features):
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        
        # Preparing the SQL insert statement
        insert_query = '''
            INSERT INTO feature_values (
                CreditScore, Age, Tenure, Balance, NumOfProducts, IsActiveMember, 
                Geography_France, Geography_Germany, Geography_Spain, BalancePerProduct, 
                Exited, data_source, version
            ) 
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        '''
        
        # Inserting the values, assuming 'features' is a dictionary with values for the columns
        # The 'version' field is populated with the current timestamp or version string
        values = (
            features['CreditScore'],
            features['Age'],
            features['Tenure'],
            features['Balance'],
            features['NumOfProducts'],
            features['IsActiveMember'],
            features['Geography_France'],
            features['Geography_Germany'],
            features['Geography_Spain'],
            features['BalancePerProduct'],
            features['Exited'] if data_source == 'train' else None,  # Exited is None for API data
            data_source,
            datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Using current timestamp as version
        )
        
        cursor.execute(insert_query, values)
        conn.commit()
        cursor.close()
        conn.close()
        print(f"✅ Data inserted for {data_source}!")

# Example usage: Inserting train data
train_data = {
    'CreditScore': 750,
    'Age': 45,
    'Tenure': 5,
    'Balance': 200000,
    'NumOfProducts': 2,
    'IsActiveMember': 1,
    'Geography_France': True,
    'Geography_Germany': False,
    'Geography_Spain': False,
    'BalancePerProduct': 100000,
    'Exited': 1  # Exited column is only filled for train data
}

insert_feature_values('train', train_data)

# Example usage: Inserting API data (Exited column is NULL)
api_data = {
    'CreditScore': 780,
    'Age': 38,
    'Tenure': 3,
    'Balance': 150000,
    'NumOfProducts': 1,
    'IsActiveMember': 1,
    'Geography_France': False,
    'Geography_Germany': True,
    'Geography_Spain': False,
    'BalancePerProduct': 150000,
    'Exited': None  # Exited is None for API data
}

insert_feature_values('api', api_data)


✅ Data inserted for train!
✅ Data inserted for api!


### Sample SQL queries

In [5]:
def list_tables():
    """Retrieve all table names in the PostgreSQL database."""
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        """
        cursor.execute(query)
        tables = cursor.fetchall()
        conn.close()
        return [table[0] for table in tables]

# Get and print table names
table_names = list_tables()
print("Tables in database:", table_names)


Tables in database: ['transformed_train_data', 'transformed_api_data', 'feature_values', 'feature_store']


In [6]:
def count_feature_values():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()

        # Fetch the latest version for train data
        cursor.execute("SELECT MAX(version) FROM feature_values WHERE data_source = 'train'")
        latest_train_version = cursor.fetchone()[0]

        # Fetch the latest version for API data
        cursor.execute("SELECT MAX(version) FROM feature_values WHERE data_source = 'api'")
        latest_api_version = cursor.fetchone()[0]

        # Count rows for train data with the latest version
        cursor.execute("SELECT COUNT(*) FROM feature_values WHERE version = %s AND data_source = 'train'", (latest_train_version,))
        train_count = cursor.fetchone()[0]

        # Count rows for API data with the latest version
        cursor.execute("SELECT COUNT(*) FROM feature_values WHERE version = %s AND data_source = 'api'", (latest_api_version,))
        api_count = cursor.fetchone()[0]

        cursor.close()
        conn.close()

        print(f"Feature Store Train Data Rows (Version {latest_train_version}): {train_count}")
        print(f"Feature Store API Data Rows (Version {latest_api_version}): {api_count}")

count_feature_values()


Feature Store Train Data Rows (Version 20250311_121740): 165034
Feature Store API Data Rows (Version 20250311_121740): 110023


In [7]:
import psycopg2
import pandas as pd

# Function to establish database connection
def connect_db():
    try:
        conn = psycopg2.connect(
            dbname='bank_churn',
            user='postgres',
            password='131412aA@',
            host='localhost',
            port='5432'
        )
        return conn
    except Exception as e:
        print("Error connecting to database:", e)
        return None

# Connect to the database
conn = connect_db()
if conn:
    # Query to count the number of rows in feature_values
    count_query = "SELECT COUNT(*) AS row_count FROM feature_values;"
    count_df = pd.read_sql(count_query, conn)
    print("Number of rows in feature_values:", count_df.iloc[0]['row_count'])
    
    # Query to fetch the first five rows from feature_values
    sample_query = "SELECT * FROM feature_values LIMIT 5;"
    sample_df = pd.read_sql(sample_query, conn)
    print("\nFirst five rows from feature_values:")
    display(sample_df)  # If running in Jupyter Notebook, display() will show the DataFrame nicely
    
    # Close the connection
    conn.close()
else:
    print("Connection to the database failed!")


Number of rows in feature_values: 275061

First five rows from feature_values:


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6288\2667823907.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count_df = pd.read_sql(count_query, conn)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6288\2667823907.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample_df = pd.read_sql(sample_query, conn)


,id,creditscore,age,tenure,balance,numofproducts,isactivemember,geography_france,geography_germany,geography_spain,balanceperproduct,exited,data_source,version
0,1,0.569231,-0.596932,3,-0.883161,2,0,True,False,False,-0.441580,0,train,20250311_121740
1,2,0.472189,-0.596932,1,-0.883161,2,1,True,False,False,-0.441580,0,train,20250311_121740
2,3,0.592899,0.258795,10,-0.883161,2,0,True,False,False,-0.441580,0,train,20250311_121740
3,4,0.363314,-0.474685,2,1.486914,1,1,True,False,False,1.486914,0,train,20250311_121740
4,5,0.682840,-0.596932,5,-0.883161,2,1,False,False,True,-0.441580,0,train,20250311_121740
